In [1]:
import pandas as pd
import numpy as np
from path import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import r2_score


In [2]:
# Loading data
file_path = Path("Resources/2022-College-Guide-Main-Rankings-clear.csv")
df_rankings = pd.read_csv(file_path)
df_rankings.head()


,UnitID,Rank,Name,8-year graduation rate,Graduation rate rank,"Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.",Graduation rate performance rank,Pell/non-Pell graduation gap,Pell graduation gap rank,Number of Pell graduates,...,% of federal work-study funds spent on service,% of federal work-study funds spent on service rank,Earns Carnegie Community Engagement Classification?,Voting engagement points,% of grads with service-oriented majors,Service-oriented majors rank,Social mobility rank,Research rank,Service rank,Unnamed: 40
0,243744,1,Stanford University (CA),0.963302,6,0.949654,187,-0.021703,46,260.6667,...,0.290903,10,NO,3,0.068361,401,3,2,187,NaN
1,215062,2,University of Pennsylvania (PA),0.964294,5,0.938620,149,-0.041917,104,376.6667,...,0.160573,71,YES,6,0.246743,209,2,9,40,NaN
2,166683,3,MA Institute of Technology (MA),0.950971,10,0.990213,334,-0.032985,72,182.6667,...,0.065477,332,NO,2,0.000000,436,5,1,324,NaN
3,186131,4,Princeton University (NJ),0.976861,2,0.947812,137,-0.052740,132,153.3333,...,0.064500,339,NO,6,0.000000,436,7,6,224,NaN
4,198419,5,Duke University (NC),0.954474,9,0.967952,267,-0.014278,36,231.6667,...,0.089523,224,YES,5,0.170164,305,12,15,20,NaN


In [3]:
df_rankings.columns

Index(['UnitID', 'Rank', 'Name', '8-year graduation rate',
       'Graduation rate rank',
       'Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.',
       'Graduation rate performance rank', 'Pell/non-Pell graduation gap',
       'Pell graduation gap rank', 'Number of Pell graduates',
       'Actual vs. predicted Pell enrollment', 'Pell performance rank',
       'Median earnings 10 years after entering college',
       'Predicted median earnings 10 years after entering college',
       'Earnings performance rank',
       'Net price of attendance for families below $75,000 income',
       'Net price rank', '% of loan principal remaining 5 years later',
       'Repayment rank', 'Predicted principal remaining',
       'Repayment rate performance rank', 'Research expenditures, in millions',
       'Research expenditures rank', 'Bachelor's to PhD rank',
       'Science & engineering PhDs awarded', 'Science & engineering PhDs rank',
       'Faculty receiving sign

In [4]:
df_rankings.dtypes

UnitID                                                                            int64
Rank                                                                              int64
Name                                                                             object
8-year graduation rate                                                          float64
Graduation rate rank                                                              int64
Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.    float64
Graduation rate performance rank                                                  int64
Pell/non-Pell graduation gap                                                    float64
Pell graduation gap rank                                                          int64
Number of Pell graduates                                                        float64
Actual vs. predicted Pell enrollment                                            float64
Pell performance rank           

In [5]:
df_rankings.count()

UnitID                                                                          442
Rank                                                                            442
Name                                                                            442
8-year graduation rate                                                          442
Graduation rate rank                                                            442
Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.    442
Graduation rate performance rank                                                442
Pell/non-Pell graduation gap                                                    442
Pell graduation gap rank                                                        442
Number of Pell graduates                                                        442
Actual vs. predicted Pell enrollment                                            442
Pell performance rank                                                       

In [6]:
# Drop last column
df_rankings.drop('Unnamed: 40', axis=1, inplace=True)

In [7]:
# Encoding Carnegie column (0=NO, 1=YES)

le = LabelEncoder()
df2_rankings = df_rankings.copy()
df2_rankings['Earns Carnegie Community Engagement Classification?'] = le.fit_transform(df2_rankings['Earns Carnegie Community Engagement Classification?'])


In [8]:
df2_rankings.head()

,UnitID,Rank,Name,8-year graduation rate,Graduation rate rank,"Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.",Graduation rate performance rank,Pell/non-Pell graduation gap,Pell graduation gap rank,Number of Pell graduates,...,ROTC rank,% of federal work-study funds spent on service,% of federal work-study funds spent on service rank,Earns Carnegie Community Engagement Classification?,Voting engagement points,% of grads with service-oriented majors,Service-oriented majors rank,Social mobility rank,Research rank,Service rank
0,243744,1,Stanford University (CA),0.963302,6,0.949654,187,-0.021703,46,260.6667,...,308,0.290903,10,0,3,0.068361,401,3,2,187
1,215062,2,University of Pennsylvania (PA),0.964294,5,0.938620,149,-0.041917,104,376.6667,...,250,0.160573,71,1,6,0.246743,209,2,9,40
2,166683,3,MA Institute of Technology (MA),0.950971,10,0.990213,334,-0.032985,72,182.6667,...,12,0.065477,332,0,2,0.000000,436,5,1,324
3,186131,4,Princeton University (NJ),0.976861,2,0.947812,137,-0.052740,132,153.3333,...,34,0.064500,339,0,6,0.000000,436,7,6,224
4,198419,5,Duke University (NC),0.954474,9,0.967952,267,-0.014278,36,231.6667,...,45,0.089523,224,1,5,0.170164,305,12,15,20


In [9]:
# Define the features set.
X = df2_rankings.copy()

# Dropping target columns and unique identifier columns, and only the Social Mobility Rank column
X = X.drop(columns=["UnitID", "Rank", "Name","Social mobility rank"])
X.head()



,8-year graduation rate,Graduation rate rank,"Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.",Graduation rate performance rank,Pell/non-Pell graduation gap,Pell graduation gap rank,Number of Pell graduates,Actual vs. predicted Pell enrollment,Pell performance rank,Median earnings 10 years after entering college,...,AmeriCorps/Peace Corps rank,ROTC rank,% of federal work-study funds spent on service,% of federal work-study funds spent on service rank,Earns Carnegie Community Engagement Classification?,Voting engagement points,% of grads with service-oriented majors,Service-oriented majors rank,Research rank,Service rank
0,0.963302,6,0.949654,187,-0.021703,46,260.6667,0.038096,122,97798,...,145,308,0.290903,10,0,3,0.068361,401,2,187
1,0.964294,5,0.938620,149,-0.041917,104,376.6667,-0.000590,211,103246,...,95,250,0.160573,71,1,6,0.246743,209,9,40
2,0.950971,10,0.990213,334,-0.032985,72,182.6667,0.076769,65,111222,...,399,12,0.065477,332,0,2,0.000000,436,1,324
3,0.976861,2,0.947812,137,-0.052740,132,153.3333,0.062997,85,95689,...,148,34,0.064500,339,0,6,0.000000,436,6,224
4,0.954474,9,0.967952,267,-0.014278,36,231.6667,0.028513,142,93115,...,39,45,0.089523,224,1,5,0.170164,305,15,20


In [10]:
# Checking datatypes to make sure there are no strings
X.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 36 columns):
 #   Column                                                                        Non-Null Count  Dtype  
---  ------                                                                        --------------  -----  
 0   8-year graduation rate                                                        442 non-null    float64
 1   Graduation rate rank                                                          442 non-null    int64  
 2   Predicted graduation rate based on % of Pell recipients, incoming SATs, etc.  442 non-null    float64
 3   Graduation rate performance rank                                              442 non-null    int64  
 4   Pell/non-Pell graduation gap                                                  442 non-null    float64
 5   Pell graduation gap rank                                                      442 non-null    int64  
 6   Number of Pell graduates          

In [11]:
# Define the target set.
y = df2_rankings["Rank"].ravel()
y[:5]


array([1, 2, 3, 4, 5], dtype=int64)

In [12]:
# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Creating a StandardScaler instance.
scaler = StandardScaler()

# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [13]:
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

Training Features Shape: (331, 36)
Training Labels Shape: (331,)
Testing Features Shape: (111, 36)
Testing Labels Shape: (111,)


## Random Forest Regressor

In [14]:
# Instantiate model with 1000 decision trees
rf_model = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf_model=rf_model.fit(X_train_scaled, y_train)

In [15]:
# Making predictions using the testing data.
rf_predictions = rf_model.predict(X_test_scaled)


In [16]:
# Evaluate the performance with r2 score
r2_rf= r2_score(y_test,rf_predictions)
print('The r2 score for this Random Forest Regressor model is:', r2_rf)

The r2 score for this Random Forest Regressor model is: 0.8561264937632609


In [17]:
# Getting accuracy data with Regressor/continuous variable model

# Calculate the absolute errors
errors = abs(rf_predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2))

# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')


Mean Absolute Error: 37.46
Accuracy: 65.72 %.


In [18]:
y_test

array([340,  46, 201, 374, 111, 355, 295, 169, 303,  84,  70,  68, 140,
       160, 222, 434, 215, 133,   7,  31, 276, 336, 263, 314,  99, 370,
       227, 273, 206, 126, 327, 250, 422, 284, 142, 192, 136, 436, 313,
       428, 168, 171, 337, 230, 204, 367, 157,  25, 292,  64, 262, 196,
         8, 254, 299,  75, 368, 361, 187,  22, 259,  93, 256,  16, 363,
       184, 249,  33,  20, 366,   3,  95, 210,  52, 182, 281, 128,  92,
       382, 349,  49, 411, 158, 224, 383, 421, 387, 351,  51, 195, 296,
        17, 274, 202,  88, 241,  21,  91, 424,  76, 137, 258,  42, 243,
       392, 309,  72, 401, 350, 233, 221], dtype=int64)

In [19]:
errors

array([ 23.315,  47.93 ,  13.379,  46.299,  42.421,  70.737,   8.61 ,
        53.199,  55.969,  37.303,  36.111,  75.137,  18.695,  48.17 ,
        10.245,  47.816, 116.393,  49.659,   5.898,   8.037,  21.405,
        28.865,  40.856,  40.613,   1.731, 145.532,  31.064,  15.099,
        22.839,  43.372,  50.765,   6.596,  52.879,   7.14 ,  58.019,
        32.968,  61.512,  73.153,   4.102,  77.591,  53.662,  45.945,
        39.416,  18.853,   4.781,  40.553,  35.185,  12.28 ,   8.668,
        87.076,  78.667,  47.782,   9.521,  25.375,  83.013,  99.344,
        25.275, 159.232,  66.773,  92.956,  27.412,   8.859,  17.161,
        16.501,   9.231,   4.31 ,   4.272,  22.577,   4.223,  35.458,
         8.   ,   1.888,  17.757,  37.034,  37.383,   5.692,  18.768,
        32.554,  30.688,  90.341,  33.989,  29.665,  15.303,  52.558,
        36.564,  45.955,  21.377,  27.192,  77.16 ,  25.906,  47.074,
        11.442,  57.872,  41.566,  16.913,  27.896,   9.7  ,  52.16 ,
        27.604,  53.

In [20]:
from sklearn import metrics
# https://machinelearningmastery.com/regression-metrics-for-machine-learning

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, rf_predictions))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, rf_predictions))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, rf_predictions)))

Mean Absolute Error: 37.461792792792785
Mean Squared Error: 2237.7207275945943
Root Mean Squared Error: 47.30455292669612


In [21]:
# Other ways of quantifying accuracy


# # Find the original feature indices 
# original_feature_indices = [feature_list.index(feature) for feature in feature_list if feature not in ['ws_1', 'prcp_1', 'snwd_1']]
# # Create a test set of the original features
# original_test_features = test_features[:, original_feature_indices]
# # Make predictions on test data using the model trained on original data
# predictions = rf.predict(original_test_features)
# # Performance metrics
# errors = abs(predictions - test_labels)
# print('Metrics for Random Forest Trained on Original Data')
# print('Average absolute error:', round(np.mean(errors), 2), 'degrees.')


In [22]:
# Calculate feature importance in the Random Forest model.
importances = rf_model.feature_importances_
importances


array([0.04828937, 0.04103564, 0.00276385, 0.07830644, 0.01786444,
       0.01929184, 0.02007115, 0.00800588, 0.00756864, 0.0129142 ,
       0.00365774, 0.07085115, 0.03598569, 0.0353207 , 0.01196478,
       0.01219878, 0.0022988 , 0.08720675, 0.01160987, 0.00915495,
       0.01107065, 0.00422943, 0.00434942, 0.16921408, 0.00277088,
       0.08124035, 0.00738835, 0.00566199, 0.00371535, 0.00363246,
       0.00639794, 0.01255611, 0.00500054, 0.00490772, 0.09020597,
       0.0512981 ])

In [23]:
# #plotting feature importances

# # Get numerical feature importances
# importances = list(rf_exp.feature_importances_)
# # List of tuples with variable and importance
# feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# # Sort the feature importances by most important first
# feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# # Print out the feature and importances 
# [print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]

# # list of x locations for plotting
# x_values = list(range(len(importances)))
# # Make a bar chart
# plt.bar(x_values, importances, orientation = 'vertical', color = 'r', edgecolor = 'k', linewidth = 1.2)
# # Tick labels for x axis
# plt.xticks(x_values, feature_list, rotation='vertical')
# # Axis labels and title
# plt.ylabel('Importance'); plt.xlabel('Variable'); plt.title('Variable Importances');

In [24]:
# Sort the features by their importance.
sorted_list=sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
sorted_df=pd.DataFrame(sorted_list, columns=['Importance', 'Feature'])
sorted_df

,Importance,Feature
0,0.169214,Faculty receiving significant awards
1,0.090206,Research rank
2,0.087207,Repayment rate performance rank
3,0.081240,Faculty accolades rank
4,0.078306,Graduation rate performance rank
5,0.070851,Earnings performance rank
6,0.051298,Service rank
7,0.048289,8-year graduation rate
8,0.041036,Graduation rate rank
9,0.035986,Net price of attendance for families below $75...


In [25]:
sorted_df.to_csv('Feature_Importances_with_ONLY_Social_mobility_rank_removed.csv')  

## Support Vector Regression

In [26]:
# Create a SVM model

# Instantiate a SVM model
from sklearn.svm import SVR

## the kernel is the most important part - try different kernels

# Fit the data

regressor = SVR(kernel = 'poly')
svm_model = regressor.fit(X_train, y_train)
svm_model


SVR(kernel='poly')

In [27]:
# Predicting the Test set results
svm_predictions = svm_model.predict(X_test)

In [28]:
from sklearn.metrics import r2_score
r2_svm=r2_score(y_test, svm_predictions)
print('The r2 score for this SVM model is:', r2_svm)


The r2 score for this SVM model is: 0.3132913566283171


In [29]:
#visualize SVR results: https://www.analyticsvidhya.com/blog/2020/03/support-vector-regression-tutorial-for-machine-learning/


## Decision Tree Regression

In [30]:
# Training the Decision Tree Regression model on the Training set
from sklearn.tree import DecisionTreeRegressor
dt_model = DecisionTreeRegressor(random_state = 0)
dt_model = dt_model.fit(X_train, y_train)
dt_model

DecisionTreeRegressor(random_state=0)

In [31]:
# Predicting the Test set results
dt_predictions = dt_model.predict(X_test)

In [32]:
# Evaluate the performance.
from sklearn.metrics import r2_score
r2_dt= r2_score(y_test,dt_predictions)
print('The r2 score for this Decision Tree model is:', r2_dt)

The r2 score for this Decision Tree model is: 0.6377319954114868


## Multiple Linear Regression

In [33]:
# Training the multiple regression model on the Training set
from sklearn.linear_model import LinearRegression
mlr_model = LinearRegression()
mlr_model = mlr_model.fit(X_train, y_train)
mlr_model

LinearRegression()

In [34]:
# Predicting the Test set results
mlr_predictions = mlr_model.predict(X_test)

In [35]:
# Evaluate the performance.
from sklearn.metrics import r2_score
r2_mlr= r2_score(y_test,mlr_predictions)
print('The r2 score for this Multiple Linear Regression model is:', r2_mlr)

The r2 score for this Multiple Linear Regression model is: 0.9593253590142109
